<a href="https://colab.research.google.com/github/AlvinManojAlex/NLP_Tamil_Hindi/blob/main/machine_translation_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting the GDrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Reading datasets collected

## 1.1 Reading the Hindi-English Parallel Corpus

In [2]:
train_en_hi = []
train_hi_hi = []

with open('/content/drive/MyDrive/corpus/train/en_hi/cvit_train.en.txt', 'r') as file:
  for line in file:

    # Appending the stripped english sentences into the list
    train_en_hi.append(line.strip())
    

with open('/content/drive/MyDrive/corpus/train/en_hi/cvit_train.hi.txt', 'r') as file:
  for line in file:
    
    # Appending the stripped hindi sentences into the list
    train_hi_hi.append(line.strip())


print(f'{len(train_en_hi)} english lines read from PIB_English_Hindi Corpus')
print(f'{len(train_hi_hi)} hindi lines read from PIB_English_Hindi Corpus')

266545 english lines read from PIB_English_Hindi Corpus
266545 hindi lines read from PIB_English_Hindi Corpus


## 1.2 Reading the Tamil-English Parallel corpus

In [3]:
train_en_ta = []
train_ta_ta = []

with open('/content/drive/MyDrive/corpus/train/en_ta/cvit_train.en.txt', 'r') as file:
  for line in file:

    # Appending the stripped english sentences into the list
    train_en_ta.append(line.strip())

with open('/content/drive/MyDrive/corpus/train/en_ta/cvit_train.ta.txt', 'r') as file:
  for line in file:

    # Appending the stripped tamil sentences into the list
    train_ta_ta.append(line.strip())

print(f'{len(train_en_ta)} english lines read from PIB_English_Tamil Corpus')
print(f'{len(train_ta_ta)} tamil lines read from PIB_English_Tamil Corpus')

115968 english lines read from PIB_English_Tamil Corpus
115968 tamil lines read from PIB_English_Tamil Corpus


In [4]:
print(train_hi_hi[0])


प्रधानमंत्री कार्यालय इंडिया टुडे कॉनक्लेव में प्रधान मंत्री के भाषण (वीडियो कान्फ्रसिंग के माध्यम से) का मूल पाठ सबसे पहले आप सभी को इस आयोजन के लिए बहुत-बहुत बधाई-शुभकामनाएं।


In [5]:
print(train_ta_ta[0])

முறையை அமல்படுத்துவதற்கு வசதியாக, பல்வேறு சரக்கு மற்றும் சேவைகளுக்கான மேல் வரி மற்றும் கூடுதல் வரியை நீக்கும் வகையில், சுங்கம் மற்றும் கலால் சட்டத்தில் திருத்தங்களைக் கொண்டுவர மத்திய அமைச்சரவை ஒப்புதல் பிரதமர் திரு.நரேந்திர மோடி தலைமையில் மத்திய அமைச்சரவைக் கூட்டம் நடைபெற்றது. இதில், கீழ்க்காணும் பரிந்துரைகளுக்கு அப்போது ஒப்புதல் அளிக்கப்பட்டது.


## 1.3 Correcting the lists so that it matches

Tamil-English corpus contains fewer sentences than the Hindi-English corpus. And to do the translation between Hindi and Tamil, we will be required to modify the corpus such that Hindi and Tamil sentences match up.

In [ ]:
# train_hi = []
# train_ta = []

# for i in range(0, len(train_en_ta)):
#   for j in range(0, len(train_en_hi)):
    
#     if train_en_ta[i] == train_en_hi[j]:
#       train_ta.append(train_ta_ta[i])
#       train_hi.append(train_hi_hi[j])
#       break
    
#     else:
#       continue

# print(len(train_hi))
# print(len(train_ta))

Saving this to a file as a new parallel corpus

In [ ]:
# with open('/content/drive/MyDrive/corpus/train/hi_ta/cvit_train.hi.txt', 'w+') as file:
#   for x in train_hi:
#     file.write(x+'\n')

# with open('/content/drive/MyDrive/corpus/train/hi_ta/cvit_train.ta.txt', 'w+') as file:
#   for x in train_ta:
#     file.write(x+'\n')

In [6]:
train_hi = []
train_ta = []

with open('/content/drive/MyDrive/corpus/train/hi_ta/cvit_train.hi.txt', 'r') as file:
  for line in file:
    train_hi.append(line.strip())

with open('/content/drive/MyDrive/corpus/train/hi_ta/cvit_train.ta.txt', 'r') as file:
  for line in file:
    train_ta.append(line.strip())

In [21]:
print(train_hi[0])

मंत्रिमण्डल मंत्रिमंडल ने सीमा एवं उत्पाद शुल्क अधिनियम में सेसों के उन्मूलन और विभिन्न वस्तुओं और सेवाओं पर अधिभार के संबंध में जीएसटी व्यवस्था को लागू करने के लिए संशोधन को अनुमोदित किया प्रधानमंत्री श्री नरेंद्र मोदी की अध्यक्षता में केंद्रीय मंत्रिमंडल ने निम्नलिखित प्रस्तावों को मंजूरी प्रदान कर दी।


# 2. Data preprocessing

Installing `tensorflow-text` and `einops` (Einstein Inspired Notation)

In [7]:
# Tensorflow package for text related operations and modules
!pip install tensorflow-text

# Installing einops for writing deep learning code better and more efficiently
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 80.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 3.0 MB/s eta 0:00:00


Importing the packages

In [17]:
import numpy as np
import pandas as pd

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import unicodedata

import tensorflow as tf
import tensorflow_text as tf_text

from tensorflow.keras import backend as K
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Every sentence should be treated as a `tf.string`, since we are trying to export this model as `tf.saved_model`.

## 2.1 Normalizing the sentences

P.S. Initially training the model without removing the bracketed words from the hindi corpus

### 2.1.1 Unicode Normalization

UNICODE Normalization is essential to maintain accuracy and efficiency in language translation models. For both Hindi and Tamil languages we will be using `NFC` (Normalization Form - C), which is a 'Canonical Decomposition followed by Canonical Composition'. This ensures that equivalent characters are represented in a consistent way.

<b>Reference:</b>

https://unicode.org/reports/tr15/

In [18]:
# Function that takes in a tensor and normalizes it according to NFC and returns the text

def unicode_to_ascii(text):
  return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

# temp = tf.constant(train_hi_hi[1])
# unicode_normalize(temp)
# print(temp)

### 2.1.2 Converting sentence to lowercase

This is done to eliminate ambiguity, since we are using English language as an intermediary so it is neccessary to convert the english corpus to its lowercase form so that the model will see 'Car' and 'car' as the same word.

### 2.1.3 Replacing some special characters

Characters like `?`, `!`, `.`, `,` and ` ` &nbsp;must not be removed from the sentence, while the other special characters must be removed. This is done by using regex to filter out the unwanted characters. Hence, we have to make a regex that does not filter out the Hindi and Tamil characters.

<b>References:</b>

https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)#:~:text=Devanagari%20is%20a%20Unicode%20block,from%20the%201988%20ISCII%20standard

https://en.wikipedia.org/wiki/Tamil_(Unicode_block)

<br>

Finally, followed by keeping a white space between the punctuations.

In [15]:
# Defining the Hindi and Tamil characters using UNICODE and then including that in the regex

# UNICODE for Hindi characters are stored continously, so we will use a loop to make our list of Hindi characters

hindi_characters = 128
hindi_unicode_shift = 0x0900

hindi_alphabets = []

for i in range(0, hindi_characters):
  hindi_alphabets.append('\\u0'+hex(hindi_unicode_shift+i)[2:])

# UNICODE for Tamil characters are not stored continuously since they have some reserved UNICODE characters in between, so we will manually add them to our list

tamil_alphabets = ['\\u0b82', '\\u0b83', '\\u0b85', '\\u0b86', '\\u0b87', '\\u0b88', '\\u0b89', '\\u0b8a', '\\u0b8e', '\\u0b8f', '\\u0b90', '\\u0b92', '\\u0b93', 
                   '\\u0b94', '\\u0b95', '\\u0b99', '\\u09b9a', '\\u0b9c', '\\u0b9e', '\\u0b9f', '\\u0ba3', '\\u0ba4', '\\u0ba8', '\\u0ba9', '\\u0baa', '\\u0bae'
                   '\\u0baf', '\\u0bb0', '\\u0bb1', '\\u0bb2', '\\u0bb3', '\\u0bb4', '\\u0bb5', '\\u0bb6', '\\u0bb7', '\\u0bb8', '\\u0bb9', '\\u0bbe', '\\u0bbf',
                   '\\u0bc0', '\\u0bc1', '\\u0bc2', '\\u0bc6', '\\u0bc7', '\\u0bc8', '\\u0bca', '\\u0bcb', '\\u0bcc', '\\u0bcd', '\\u0bd0', '\\u0bd7', '\\u0be6',
                   '\\u0be7', '\\u0be8', '\\u0be9', '\\u0bea', '\\u0beb', '\\u0bec', '\\u0bed', '\\u0bee', '\\u0bef', '\\u0bf0', '\\u0bf1', '\\u0bf2', '\\u0bf3',
                   '\\u0bf4', '\\u0bf5', '\\u0bf6', '\\u0bf7', '\\u0bf8', '\\u0bf9', '\\u0bfa']

In [19]:
# Function that takes in a tensor and keeps `?`, `!`, `.`, `,`, ` ` as such and replaces the other special characters with ``
# After that a white space is kept between the 'chosen' punctuations
# Account for regex with Hindi and Tamil
# Also account for more than 1 whitespace being generated

import re

def punctuate(text):
  text = unicode_to_ascii(text)
  regex_pattern = r"[^,.?! A-Za-z\u0900-\u097F\u0b82\u0b83\u0b85\u0b86\u0b87\u0b88\u0b89\u0b8a\u0b8e\u0b8f\u0b90\u0b92\u0b93\u0b94\u0b95\u0b99\u09b9a\u0b9c\u0b9e\u0b9f\u0ba3\u0ba4\u0ba8\u0ba9\u0baa\u0bae\u0baf\u0bb0\u0bb1\u0bb2\u0bb3\u0bb4\u0bb5\u0bb6\u0bb7\u0bb8\u0bb9\u0bbe\u0bbf\u0bc0\u0bc1\u0bc2\u0bc6\u0bc7\u0bc8\u0bca\u0bcb\u0bcc\u0bcd\u0bd0\u0bd7\u0be6\u0be7\u0be8\u0be9\u0bea\u0beb\u0bec\u0bed\u0bee\u0bef\u0bf0\u0bf1\u0bf2\u0bf3\u0bf4\u0bf5\u0bf6\u0bf7\u0bf8\u0bf9\u0bfa]+"
  processed_string = re.sub(regex_pattern, "", text)
  processed_string = re.sub('([,.?!])', r' \1', processed_string)
  return processed_string.strip()

# def tamil_punctuate(text):
#   regex_pattern = r"[^,.?! \u0b82\u0b83\u0b85\u0b86\u0b87\u0b88\u0b89\u0b8a\u0b8e\u0b8f\u0b90\u0b92\u0b93\u0b94\u0b95\u0b99\u09b9a\u0b9c\u0b9e\u0b9f\u0ba3\u0ba4\u0ba8\u0ba9\u0baa\u0bae\u0baf\u0bb0\u0bb1\u0bb2\u0bb3\u0bb4\u0bb5\u0bb6\u0bb7\u0bb8\u0bb9\u0bbe\u0bbf\u0bc0\u0bc1\u0bc2\u0bc6\u0bc7\u0bc8\u0bca\u0bcb\u0bcc\u0bcd\u0bd0\u0bd7\u0be6\u0be7\u0be8\u0be9\u0bea\u0beb\u0bec\u0bed\u0bee\u0bef\u0bf0\u0bf1\u0bf2\u0bf3\u0bf4\u0bf5\u0bf6\u0bf7\u0bf8\u0bf9\u0bfa]+"
#   processed_string = re.sub(regex_pattern, "", text)
#   processed_string = re.sub('([,.?!])', r' \1', processed_string)
#   return processed_string

# def english_punctuate(text):
#   text = tf.strings.regex_replace(text, '[^ a-z.?!,]', '')
#   text = tf.strings.regex_replace(text, '[.,?!]', r' \0 ')
#   text = tf.strings.strip(text)
#   return text

# text = train_en_ta[1]+'@'
# print(text)
# text = punctuate(text)
# print(text)
# temp = tf.constant(text)
# print(temp)
# temp = unicode_normalize(temp)

# text = train_ta_ta[1]
# print(text)
# text = tamil_punctuate(text)
# print(text)
# temp = tf.constant(text)
# print(temp)
# temp = unicode_normalize(temp)
# print(temp)


### 2.1.4 Adding START and END tokens

This helps the model in understanding where the beginning and end of a sequence is. The model we will be making, operates on sequences of fixed length, so adding these tokens will help the model mark where the sentence begins or ends. 

Thus, improving model performance and translation quality.

In [ ]:
# # Adding START and END Tokens to the sentence.

# def start_end(text):

#   text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
#   return text

# # temp = tf.constant(train_ta_ta[1])
# # print(temp)
# # temp = start_end(temp)
# # print(temp)

### 2.1.5 Compiling these functions into one

This is done so that we can pass one single argument for the standardization parameter in the tf.keras.layers.TextVectorization

In [ ]:
# def compile_normalize(text):
  
#   # some error with decoding when we make the TextVectorization layer
#   text = punctuate(text.numpy().decode()[0])
#   text = unicode_normalize(text)
#   text = lowercase(text)
#   text = start_end(text)
#   return text


## 2.2 Loading the dataset

We will load the dataset into a pandas dataframe, so that we can easily apply the preprocessing functions

In [20]:
df = pd.DataFrame({'INPUT_COLUMN': train_hi, 'TARGET_COLUMN': train_ta})

input_data = df['INPUT_COLUMN'].apply(lambda x : punctuate(x)).tolist()
target_data = df['TARGET_COLUMN'].apply(lambda x : punctuate(x) + ' <eos>').tolist()
target_input_data = df['TARGET_COLUMN'].apply(lambda x : '<sos> ' + punctuate(x)).tolist()

print(input_data[:5])
print(target_data[:5])
print(target_input_data[:5])

['मतरिमणडल मतरिमडल न सीमा एव उतपाद शलक अधिनियम म ससो क उनमलन और विभिनन वसतओ और सवाओ पर अधिभार क सबध म जीएसटी वयवसथा को लाग करन क लिए सशोधन को अनमोदित किया परधानमतरी शरी नरदर मोदी की अधयकषता म कदरीय मतरिमडल न निमनलिखित परसतावो को मजरी परदान कर दी।', 'सीमा शलक अधिनियम , म सशोधन  .', 'सीमा शलक टरिफ अधिनियम , म सशोधन  .', 'कदरीय उतपाद शलक टरिफ अधिनियम अपील ,   .', 'मतरिमणडल कदरीय मतरिमडल न सटारटपस क लिए निधि सथापित करन को मजरी दी परधानमतरी शरी नरदर मोदी की अधयकषता म कदरीय मतरिमडल न सटारटपस क लिए निधि सथापित करन क लिए निमनलिखित परसतावो को अनमति परदान कर दी ह।']
['முறையை அமலபடுததுவதறகு வதியாக , பலவேறு ரககு மறறும ேவைகளுககான மேல வரி மறறும கூடுதல வரியை நககும வகையில , ுஙகம மறறும கலால டடததில திருததஙகளைக கொணடுவர மததிய அமைரவை ஒபபுதல பிரதமர திரு .நரேநதிர மோடி தலைமையில மததிய அமைரவைக கூடடம நடைபெறறது . இதில , கழககாணும பரிநதுரைகளுககு அபபோது ஒபபுதல அளிககபபடடது . <eos>', 'மேறகணட பரிநதுரைகளால , கழககாணும பலனகள கிடைககும ுஙகஙகள டடம ல ஏ , பி பிரிவுகள ேரககபபடுகினறன . <eos>', 'ஏனெனில , இநத டடஙகளை ிறபபான முறைய

In [22]:
# Deleting to save memory

del df

## 2.3 Tokenizing and Processing the text data

--> Tokenize the data, i.e., to convert the text into sequence of integers.

--> Extract the sequence of integers from the text

--> Finding the maximum length of input and output sequences

In [24]:
# MAX_VOCAB_SIZE = 20000
# EMBEDDING_DIM = 128
# HIDDEN_DIM=1024 #512

# BATCH_SIZE = 64  # Batch size for training.
# EPOCHS = 10  # Number of epochs to train for.

# ATTENTION_FUNC='general'

# Making a tokenizer for the input texts and fit them

tokenizer_inputs = Tokenizer(num_words = 20000, filters='')
tokenizer_inputs.fit_on_texts(input_data)

# Tokenize and transforming texts into sequence of integers

input_sequences = tokenizer_inputs.texts_to_sequences(input_data)

# Calculating the max length

input_max_len = max(len(s) for s in input_sequences)
print(f'The Maximum input length is {input_max_len}')

The Maximum input length is 256


In [25]:
# How input tokenized text looks like

print(input_data[0])
print(input_sequences[0])

मतरिमणडल मतरिमडल न सीमा एव उतपाद शलक अधिनियम म ससो क उनमलन और विभिनन वसतओ और सवाओ पर अधिभार क सबध म जीएसटी वयवसथा को लाग करन क लिए सशोधन को अनमोदित किया परधानमतरी शरी नरदर मोदी की अधयकषता म कदरीय मतरिमडल न निमनलिखित परसतावो को मजरी परदान कर दी।
[446, 117, 10, 482, 24, 719, 952, 400, 2, 1, 2675, 5, 96, 769, 5, 451, 13, 7750, 1, 282, 2, 542, 289, 6, 293, 15, 1, 11, 414, 6, 2753, 21, 20, 23, 671, 54, 4, 186, 2, 108, 117, 10, 892, 2374, 6, 110, 94, 26, 379]


In [27]:
# Creating a tokenizer for the output and fit it to them, including the data with both <eos> and <sos> tags

tokenizer_outputs = Tokenizer(num_words = 20000, filters='')
tokenizer_outputs.fit_on_texts(target_data)
tokenizer_outputs.fit_on_texts(target_input_data)

# Tokenize and transform output texts to sequence of integers

target_sequences = tokenizer_outputs.texts_to_sequences(target_data)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_input_data)

# Finding the maximum length for target sequence

target_max_len = max(len(s) for s in target_sequences)
print(f'The Maximum target length is {target_max_len}')

The Maximum target length is 231


In [28]:
# How target tokenized texts look like

print(target_data[0])
print(target_sequences[0])
print(target_input_data[0])
print(target_sequences_inputs[0])

முறையை அமலபடுததுவதறகு வதியாக , பலவேறு ரககு மறறும ேவைகளுககான மேல வரி மறறும கூடுதல வரியை நககும வகையில , ுஙகம மறறும கலால டடததில திருததஙகளைக கொணடுவர மததிய அமைரவை ஒபபுதல பிரதமர திரு .நரேநதிர மோடி தலைமையில மததிய அமைரவைக கூடடம நடைபெறறது . இதில , கழககாணும பரிநதுரைகளுககு அபபோது ஒபபுதல அளிககபபடடது . <eos>
[711, 8625, 2445, 2, 29, 566, 5, 6375, 1036, 166, 5, 296, 3486, 6834, 35, 2, 5828, 5, 5993, 1855, 8626, 2344, 8, 51, 50, 7, 9, 285, 34, 203, 8, 341, 466, 723, 1, 165, 2, 3974, 947, 50, 596, 1, 3]
<sos> முறையை அமலபடுததுவதறகு வதியாக , பலவேறு ரககு மறறும ேவைகளுககான மேல வரி மறறும கூடுதல வரியை நககும வகையில , ுஙகம மறறும கலால டடததில திருததஙகளைக கொணடுவர மததிய அமைரவை ஒபபுதல பிரதமர திரு .நரேநதிர மோடி தலைமையில மததிய அமைரவைக கூடடம நடைபெறறது . இதில , கழககாணும பரிநதுரைகளுககு அபபோது ஒபபுதல அளிககபபடடது .
[4, 711, 8625, 2445, 2, 29, 566, 5, 6375, 1036, 166, 5, 296, 3486, 6834, 35, 2, 5828, 5, 5993, 1855, 8626, 2344, 8, 51, 50, 7, 9, 285, 34, 203, 8, 341, 466, 723, 1, 165, 2, 3974, 947, 50, 596, 1]


## 2.4 Mapping to make vocabularies

Using the tokenizer we have created previously we can retrieve the vocabularies, one to match word to integer (word2idx) and a second one to match the integer to the corresponding word (idx2word).

In [29]:
# get the word to index mapping for input language

word2idx_inputs = tokenizer_inputs.word_index
print(f'Found {len(word2idx_inputs)} unique input tokens.')

# get the word to index mapping for output language

word2idx_outputs = tokenizer_outputs.word_index
print(f'Found {len(word2idx_outputs)} unique output tokens.')

# store number of output and input words for later
# Adding 1 since indexing starting from 1

num_words_output = len(word2idx_outputs) + 1
num_words_inputs = len(word2idx_inputs) + 1

# map indexes back into real words so that we can view the results

idx2word_inputs = {v:k for k, v in word2idx_inputs.items()}
idx2word_outputs = {v:k for k, v in word2idx_outputs.items()}

Found 45894 unique input tokens.
Found 116293 unique output tokens.


## 2.5 Padding the sentences 

We need to pad zeroes at the end of the sequences so that all the sequences have the same length, for training.

In [30]:
# pad the input sequences

encoder_inputs = pad_sequences(input_sequences, maxlen=input_max_len, padding='post')
print("encoder_inputs.shape:", encoder_inputs.shape)
print("encoder_inputs[0]:", encoder_inputs[0])

# pad the decoder input sequences

decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=target_max_len, padding='post')
print("decoder_inputs[0]:", decoder_inputs[0])
print("decoder_inputs.shape:", decoder_inputs.shape)

# pad the target output sequences

decoder_targets = pad_sequences(target_sequences, maxlen=target_max_len, padding='post')

encoder_inputs.shape: (73355, 256)
encoder_inputs[0]: [ 446  117   10  482   24  719  952  400    2    1 2675    5   96  769
    5  451   13 7750    1  282    2  542  289    6  293   15    1   11
  414    6 2753   21   20   23  671   54    4  186    2  108  117   10
  892 2374    6  110   94   26  379    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0   

## 2.6 Making Batch Data generator

We will train the model on batches, i.e., group of sentences, so we need to create a Dataset using the tf.data library and the function batch_on_slices on the input and output sequences.

In [31]:
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((encoder_inputs, decoder_inputs, decoder_targets))
dataset = dataset.shuffle(len(input_data)).batch(BATCH_SIZE, drop_remainder=True)

# 3. Making the Model

We will be making an Encoder-Decoder model with RNNs (Recurrent Neural Networks)

--> Encoder: Layers of recurrent units where in each time step, receive an input token, collects relevant information and produce a hidden state. Depending on the type of RNN, in our example an LSTM, the unit "mixes" the current hidden state and the input and return an output, discarded, and a new hidden state.

--> Encoder Vector: It is the last hidden state of the encoder and it tries to contain as much of the useful input information as possible to help the decoder get the best results. It is only information from the input that the decoder will get.

--> Decoder: Layers of recurrent units, i.e. LSTMs, where each unit produces an output at a time step 't'. The hidden state of the first unit is the encoder vector and the rest of units accept the hidden state from the previous unit. The output is calculated using a softmax function to obtain a probability for every token in the output vocabulary.

# 4. Training the Model

# 5. Evaluating accuracy